In [1]:
%load_ext tensorboard

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [2]:
tf.random.set_seed(22)
np.random.seed(22)

batch_size = 128
total_words = 10000
max_review_len = 80
embedding_len = 100

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=total_words)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(10000).batch(batch_size, drop_remainder=True)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

print('x_train_shape:', x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print('x_test_shape:', x_test.shape)

sample = next(iter(test_data))
print(sample[0].shape)

x_train_shape: (25000, 80) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
x_test_shape: (25000, 80)
(128, 80)


In [6]:
class LSTM_Build(tf.keras.Model):
    def __init__(self, units):
        super(LSTM_Build, self).__init__()
        
        self.state0 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        self.state1 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        
        
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len,
                                                   input_length=max_review_len)
        self.RNNCell0 = tf.keras.layers.LSTMCell(units, dropout=0.5)
        self.RNNCell1 = tf.keras.layers.LSTMCell(units, dropout=0.5)
        self.outlayer = tf.keras.layers.Dense(1)
        
    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1):
            out0, state0 = self.RNNCell0(word, state0, training)
            out1, state1 = self.RNNCell1(out0, state1, training)
            
        x = self.outlayer(out1)
        prob = tf.sigmoid(x)
        
        return prob

In [7]:
import time
units = 64
epochs = 4
t0 = time.time()

log_dir = 'img/log7-2/'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                      histogram_freq=1, profile_batch=0)

model = LSTM_Build(units)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              experimental_run_tf_function=False)

model.fit(train_data, epochs=epochs, validation_data=test_data, callbacks=[tensorboard_callback], validation_freq=2)

Epoch 1/4
195/195 [==============================] - 56s 161ms/step - loss: 0.4769 - accuracy: 0.7607
Epoch 2/4
195/195 [==============================] - 57s 294ms/step - loss: 0.3128 - accuracy: 0.8700 - val_loss: 0.3662 - val_accuracy: 0.8375
Epoch 3/4
195/195 [==============================] - 35s 181ms/step - loss: 0.2615 - accuracy: 0.8956
Epoch 4/4
195/195 [==============================] - 50s 256ms/step - loss: 0.2207 - accuracy: 0.9153 - val_loss: 0.4006 - val_accuracy: 0.8330


In [8]:
print('훈련 데이터셋 평가...')
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print('loss={:.4f}, accuracy: {:.4f}%'.format(loss, accuracy*100))
print('테스트 데이터셋 평가...')
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print('loss={:.4f}, accuracy: {:.4f}%'.format(loss, accuracy*100))

t1 = time.time()
print('시간:', t1-t0)

훈련 데이터셋 평가...
loss=0.1472, accuracy: 95.4247%
테스트 데이터셋 평가...
loss=0.4006, accuracy: 83.3013%
시간: 423.80303502082825


![accuracy](img/accuracy-2.png)